In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
import re
import time
import sys

In [12]:
chrome_path = "C:\\Users\\wmk51\\python-project\\chromedriver.exe"
options = webdriver.ChromeOptions()
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument("no-sandbox")
options.add_argument("disable-gpu")
options.add_argument("--lang=ko_KR")
options.add_argument('user-agent=' + user_agent)
driver = webdriver.Chrome(options=options)
url = "https://www.musinsa.com/search/musinsa/goods?q=%EB%8B%88%ED%8A%B8&list_kind=small&sortCode=pop&sub_sort=&page=1&display_cnt=0&saleGoods=&includeSoldOut=&setupGoods=&popular=&category1DepthCode=&category2DepthCodes=&category3DepthCodes=&selectedFilters=&category1DepthName=&category2DepthName=&brandIds=&price=&colorCodes=&contentType=&styleTypes=&includeKeywords=&excludeKeywords=&originalYn=N&tags=&campaignId=&serviceType=&eventType=&type=&season=&measure=&openFilterLayout=N&selectedOrderMeasure=&shoeSizeOption=&d_cat_cd=&attribute=&plusDeliveryYn="
driver.get(url)
driver.set_window_size(1920,1080)
driver.maximize_window()

In [13]:
WebDriverWait(driver, 8).until(EC.element_to_be_clickable((By.CLASS_NAME, "lcWnUT"))).click()

time.sleep(2)
html = driver.page_source
soup = BeautifulSoup(html,"html.parser")

#브랜드 이름 리스트
brand_name_list=[]
### 상품명 이름 리스트
men_prod_name_list=[]
women_prod_name_list=[]

### 할인전 가격 리스트
men_before_discount_list=[]
women_before_discount_list=[]

### 할인 후 가격 리스트
men_after_discount=[]
women_after_discount=[]



In [14]:

items = soup.find_all("div",attrs={"class" : "li_inner"})[:30]
time.sleep(1)

for item in items:    
    #브랜드 가져오기
    try:
        brand_name = item.find("p",attrs={"class":"item_title"}).find("a").get_text().replace(' ', '').replace('\n', '')
        brand_name_list.append(brand_name)
        
    except:
        brand_name="데이터 없음"
        brand_name_list.append(brand_name)
    
    #상품명 가져오기
    try:
        item_name = item.find("p",attrs={"class" : "list_info"}).find("a").get_text().replace(' ', '').replace('\n', '')
        men_prod_name_list.append(item_name)
    except:
        item_name = "데이터 없음"
        men_prod_name_list.append(item_name)
    
    #할인 전 가격과 할인 후 가격 가져오기
    #할인을 하지 않은 상품도 있으니 <del>을 이용해서 나눠준다.
    
    #할인 전 가격
    try :
        price_before_discount = item.find("p",attrs={"class" : "price"}).find("del").get_text()
        price_before_discount = int(re.sub(r'[^0-9]', '', price_before_discount))
        men_before_discount_list.append(price_before_discount)
        
    except:
        price_before_discount = item.find("p",attrs={"class" : "price"}).get_text().strip()
        price_before_discount = int(re.sub(r'[^0-9]', '', price_before_discount))
        men_before_discount_list.append(price_before_discount)
        
    #할인 후 가격
    
    try:
        price_after_discount = item.find("p",attrs={"class" : "price"}).get_text().strip().split(' ')[1]
        price_after_discount = int(re.sub(r'[^0-9]', '', price_after_discount))
        men_after_discount.append(price_after_discount)
    except:
       
        men_after_discount.append(price_before_discount)
    

In [15]:
print("브랜드명  ",len(brand_name_list))
print("상품명  ",len(men_prod_name_list))
print("할인전  ",len(men_before_discount_list))
print("할인후  ",len(men_after_discount))

브랜드명   30
상품명   30
할인전   30
할인후   30


In [16]:
import pandas as pd
men_price_df = pd.DataFrame({
                            "브랜드명": brand_name_list,
                            "상품명":men_prod_name_list,
                            "할인전 가격": men_before_discount_list,
                            "할인후 가격" : men_after_discount})

In [17]:
men_price_df[:3]

,브랜드명,상품명,할인전 가격,할인후 가격
0,데이,3/19배송[ONLYOUTLET]시그니처라운드니트(10COLOR),65900,19400
1,미니멀코드,소프트필링세미오버라운드니트초코브라운,59000,33000
2,맨인스토어,3/22배송꽈배기후드반집업니트_Gray,139900,39900


In [18]:
import math
men_price_df["할인율(%)"] = ((men_price_df['할인전 가격'] - men_price_df['할인후 가격']) / men_price_df['할인전 가격']) * 100
men_price_df["할인율(%)"] = men_price_df["할인율(%)"].apply(lambda x: math.ceil(x))



# men_price_df = men_price_df.loc[men_price_df['할인율(%)'] == 100, '할인율(%)'] = 0


# 할인율이 100인 경우에만 할인된 가격을 0으로 변경
# for index, row in men_price_df.iterrows():
#     if row['할인율(%)'] == 100:
#         men_price_df.at[index, '할인율(%)'] = 0

In [19]:
men_price_df[:5]

,브랜드명,상품명,할인전 가격,할인후 가격,할인율(%)
0,데이,3/19배송[ONLYOUTLET]시그니처라운드니트(10COLOR),65900,19400,71
1,미니멀코드,소프트필링세미오버라운드니트초코브라운,59000,33000,45
2,맨인스토어,3/22배송꽈배기후드반집업니트_Gray,139900,39900,72
3,언리미트,소프트니트(U23DTNT161),69000,16900,76
4,키뮤어,[2-PACK](하프집업+크루넥선택)코튼워셔블하찌니트,128000,43500,67


In [21]:
men_price_df
men_price_df.to_csv('musinsa_men.csv', index=False,encoding='utf-8-sig') 

In [24]:
import pandas as pd
men_price_df = pd.read_csv('./musinsa_men.csv')
men_price_df

,브랜드명,상품명,할인전 가격,할인후 가격,할인율(%)
0,데이,3/19배송[ONLYOUTLET]시그니처라운드니트(10COLOR),65900,19400,71
1,미니멀코드,소프트필링세미오버라운드니트초코브라운,59000,33000,45
2,맨인스토어,3/22배송꽈배기후드반집업니트_Gray,139900,39900,72
3,언리미트,소프트니트(U23DTNT161),69000,16900,76
4,키뮤어,[2-PACK](하프집업+크루넥선택)코튼워셔블하찌니트,128000,43500,67
5,예일,HERITAGEDANCABLEKNITHOODIEZIP-UPIVORY,79000,55300,30
6,로열클럽홀리데이,로고반집업니트맨투맨네이비,89000,59800,33
7,디미트리블랙,2WAY헤비웨이트케이블니트후드집업_오트밀,66000,49900,25
8,블랙엔드,3/28배송BRANCHLOGOKNIT-KHAKI,79000,55300,30
9,어반디타입,컬러블록래글런니트_버건디_KN124,59000,42900,28


In [25]:
men_avg_price=men_price_df["할인후 가격"].mean()
men_avg_price


49410.666666666664

In [26]:
sale_list = []
men_sale_price_list = men_price_df["할인후 가격"].to_list()
for price in men_sale_price_list:
    if(price<men_avg_price):
        sale_list.append(True)
    else:
        sale_list.append(False)
   
men_price_df["super_sale"] = sale_list    

In [28]:
men_price_df.to_csv('musinsa_men.csv', index=False,encoding='utf-8-sig') 